In [2]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [3]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [4]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [6]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [7]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [9]:
print("emb matrix shape", emb_matrix.shape)
print("vocab len", len(vocab))


emb matrix shape (10001, 300)
vocab len 10000


In [10]:
imp.reload(util)

seqlen = 128
stride = 96
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride, numfiles=100)



  0%|          | 50/22860 [00:00<00:46, 489.87it/s]


In [14]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                           trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [15]:
Lemb = L_emb
emb_x = Lemb(input_x)
#emb_y = Lemb(input_y)

In [16]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
#concat_y = Lconcat([emb_y, resh_yu])

In [17]:
lstm = K.layers.CuDNNGRU(256, return_sequences=True)(concat_x)

dense = K.layers.Dense(dim, activation="softmax")(lstm)

#dense = K.layers.BatchNormalization()(lstm)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)



In [19]:
model = K.Model(inputs=[input_x, input_xu],
               outputs=[dense])

In [21]:
opt = K.optimizers.Adam(lr=0.1)
model.compile(opt, loss='categorical_crossentropy')

In [25]:
Yu.shape

(617, 128)

In [ ]:
batch_size=256

def data_generator(X, Xu, Y, Yu, vocab_size=10000):
    for i in range(0, X.shape[0], batch_size):
        start, end = i, i+batch_size
        bX  = X[start:end]
        bXu = Xu[start:end]
        
        bY = np.concatenate(K.utils.to_categorical(num_classes=vocab_size),
                            np.expand_dims(Yu, axis=2), axis=2)
        yield [[bX, bXu], [bY]]

In [37]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=1000)


Epoch 1/1000
617/617 [==============================] - 2s 4ms/step - loss: nan
Epoch 2/1000
617/617 [==============================] - 0s 710us/step - loss: nan
Epoch 3/1000
617/617 [==============================] - 0s 710us/step - loss: nan
Epoch 4/1000
617/617 [==============================] - 0s 701us/step - loss: nan
Epoch 5/1000
617/617 [==============================] - 0s 718us/step - loss: nan
Epoch 6/1000
617/617 [==============================] - 0s 724us/step - loss: nan
Epoch 7/1000
617/617 [==============================] - 0s 714us/step - loss: nan
Epoch 8/1000
617/617 [==============================] - 0s 735us/step - loss: nan
Epoch 9/1000
617/617 [==============================] - 0s 743us/step - loss: nan
Epoch 10/1000
617/617 [==============================] - 0s 747us/step - loss: nan
Epoch 11/1000


KeyboardInterrupt: 

In [274]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="mean_squared_error")

In [275]:
from sklearn import preprocessing as pp
norm_emb_matrix =  pp.normalize(Lemb.get_weights()[0], axis=1)

In [276]:
i = 7
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

iterations = 128
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    #print(tX)
    xv, xuv = 0, 0.0
    z = predictor.predict([tX, tXu])
    if z[0, -1, -1] > 0.1:
        xuv = 1.0
        results.append("<UNK>")
    else:
        we = np.transpose(pp.normalize(np.reshape(z[0, -1, :300], (1,300))))
#        print(np.linalg.norm(we))
#        scores = np.matmul(norm_emb_matrix, we)
#        print(scores)
#        print(np.min(scores))
#        break
        xv = np.argmax(np.matmul(norm_emb_matrix, we))
        if xv:
            results.append(words[xv])
        else:
            results.append("<EMPTY>")
    gen.append(xv)
    genu.append(xuv)
    print(results[-1])

<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>
<UNK>


In [66]:
print(np.min(z), np.max(z))

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'